In [ ]:
# Plotting for synthetic experiments

import inverse_cai.paper_plotting
import pathlib

import importlib
importlib.reload(inverse_cai.paper_plotting)

N1_PATH = pathlib.Path("../exp/saved_results/30_synth_regularisation_n/n_1")
N3_PATH = pathlib.Path("../exp/saved_results/30_synth_regularisation_n/n_3")
N5_PATH = pathlib.Path("../exp/saved_results/01_synth_orth_6seeds")
N10_PATH = pathlib.Path("../exp/saved_results/30_synth_regularisation_n/n_10")


n1_df = inverse_cai.paper_plotting.get_metrics_dict(N1_PATH, "train")
n3_df = inverse_cai.paper_plotting.get_metrics_dict(N3_PATH, "train")
n5_df = inverse_cai.paper_plotting.get_metrics_dict(N5_PATH, "train")
n10_df = inverse_cai.paper_plotting.get_metrics_dict(N10_PATH, "train")

# remove the default model from the n=5 results, not relevant here
n5_df.pop("chatgpt_fn_noinstruction")

combined_data = {
    "n=1": {"": n1_df},
    "n=3": {"": n3_df},
    "n=5": {"": n5_df},
    "n=10": {"": n10_df},
}

# also sets the model order
model_name_map = {
    'chatgpt_fn_noinstruction': "Default",
    'chatgpt_fn_constitutional_base_neutral_v1': "Constitutional",
}

pathlib.Path("figures").mkdir(exist_ok=True)
inverse_cai.paper_plotting.plot_data(
    combined_data,
    "figures/0016_experiments_synthetic_orth_n_gpt35turbo.png",
    #model_name_map=model_name_map,
    add_constitutions=False,
    y_label="Agreement (%)\nwith original data",
    legend_remove=True,
)

# For the latex table, we want to
# - include the model name ("GPT-3.5 Turbo")
# - use "Unaligned" as the dataset name
# - append the number of seeds to the annotator name.
assert len(n1_df) == len(n3_df) == len(n5_df) == len(n10_df) == 1
table_combined_data = {
    "Unaligned": {
        "GPT-3.5 Turbo": {
            "Constitutional (n=1)": n1_df["chatgpt_fn_constitutional_base_neutral_v1"],
            "Constitutional (n=3)": n3_df["chatgpt_fn_constitutional_base_neutral_v1"],
            "Constitutional (n=5)": n5_df["chatgpt_fn_constitutional_base_neutral_v1"],
            "Constitutional (n=10)": n10_df["chatgpt_fn_constitutional_base_neutral_v1"],
        }
    }
}
num_seeds = len(n1_df["chatgpt_fn_constitutional_base_neutral_v1"]["values"])
latex_table = inverse_cai.paper_plotting.generate_latex_table(table_combined_data, f"Results for the sensitivity study on parameter $n$ (rules per constitution) on synthetic data. Averaged over {num_seeds}~random seeds.", "tab:synthetic_param_n_num_results", "appendix/numerical_results/synthetic_param_n.tex")
print(latex_table)